<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/producedwatertreatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Simulation of produced water treatment
#@markdown Introduction to process simualtion of produced water treatment
%%capture
!pip install neqsim
from neqsim.process import openprocess
import neqsim
from neqsim.thermo.thermoTools import *
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

#Litterature

https://vmecompanies.com/produced-water-treatment/

https://www.filtsep.com/water-and-wastewater/features/oil-and-gas-water-treatment-in-oil-and-gas/

https://www.filtsep.com/water-and-wastewater/features/oil-and-gas-treatment-and-discharge-of-produced/


#Simulation of produced water treatment using NeqSim
THe CPA-EoS in NeqSIm is well suted for simulating solubilities of oil in water and water in oil for a varity of reservoir fluids and process conditions.
In the following example we will simulate a typical topside separation and water treatment process using NeqSim.

#1. Defining the well stream

In [2]:
# Create a gas-condensate fluid
gascondensate = {'ComponentName':  ['water', "nitrogen", "CO2", "methane", "ethane", "propane", "i-butane", "n-butane", "i-pentane", "n-pentane", "n-hexane", "C7", "C8", "C9", "C10", "C11", "C12", "C13", "C14", "C15", "C16", "C17", "C18", "C19", "C20"], 
        'MolarComposition[-]':  [0.0, 0.53, 3.3, 72.98, 7.68, 4.1, 0.7, 1.42, 0.54, 0.67, 0.85, 1.33, 1.33, 0.78, 0.61, 0.42, 0.33, 0.42, 0.24, 0.3, 0.17, 0.21, 0.15, 0.15, 0.8], 
        'MolarMass[kg/mol]': [None, None,None, None,None,None,None,None,None,None,None,0.0913, 0.1041, 0.1188, 0.136, 0.150, 0.164, 0.179, 0.188, 0.204, 0.216, 0.236, 0.253, 0.27, 0.391],
        'RelativeDensity[-]': [None, None,None, None,None,None,None,None,None,None,None, 0.746, 0.768, 0.79, 0.787, 0.793, 0.804, 0.817, 0.83, 0.835, 0.843, 0.837, 0.84, 0.85, 0.877]
  } 

gascondensatedf = pd.DataFrame(gascondensate) 
print("Gas Condensate Fluid:\n")
print(gascondensatedf.head(30).to_string())
gascondensateFluid = fluid_df(gascondensatedf, lastIsPlusFraction=True).autoSelectModel()

Gas Condensate Fluid:

   ComponentName  MolarComposition[-]  MolarMass[kg/mol]  RelativeDensity[-]
0          water                 0.00                NaN                 NaN
1       nitrogen                 0.53                NaN                 NaN
2            CO2                 3.30                NaN                 NaN
3        methane                72.98                NaN                 NaN
4         ethane                 7.68                NaN                 NaN
5        propane                 4.10                NaN                 NaN
6       i-butane                 0.70                NaN                 NaN
7       n-butane                 1.42                NaN                 NaN
8      i-pentane                 0.54                NaN                 NaN
9      n-pentane                 0.67                NaN                 NaN
10      n-hexane                 0.85                NaN                 NaN
11            C7                 1.33             0.0

#2. Defining the water stream



In [3]:
# Create a gas-condensate fluid
water = {'ComponentName':  ["water"], 
        'MolarComposition[-]':  [1.0]
  } 

waterdf = pd.DataFrame(water) 
print("Water:\n")
print(waterdf.head(30).to_string())
waterFluid = fluid_df(waterdf)

Water:

  ComponentName  MolarComposition[-]
0         water                  1.0


#2. Creating the separation process

In [7]:
from neqsim.thermo import fluid
from neqsim.process import clearProcess, stream, valve, runProcess, viewProcess, mixer, recycle,separator3phase, separator


clearProcess()
reservoirFluidStream = stream(gascondensateFluid)
reservoirFluidStream.setFlowRate(5.3, 'MSm3/day')
reservoirFluidStream.setPressure(85.2, 'bara')
reservoirFluidStream.setTemperature(65.1, 'C')

waterStream = stream(waterFluid)
waterStream.setFlowRate(1090500.0, 'kg/day')
waterStream.setPressure(85.2, 'bara')
waterStream.setTemperature(65.1, 'C')

feedMixer = mixer()
feedMixer.addStream(reservoirFluidStream)
feedMixer.addStream(waterStream)

inletSeparator = separator3phase(feedMixer.getOutStream(), "1st stage separator")

waterDpValve = valve(inletSeparator.getWaterOutStream(), 1.01325, 'water depressurising valve')

waterDegasingDrum = separator(waterDpValve.getOutStream(), "water degassing drum")

gasToFlare = waterDegasingDrum.getGasOutStream()

waterToSea = waterDegasingDrum.getLiquidOutStream()

runProcess()

print('gas to flare ', gasToFlare.getFlowRate('Sm3/hr'), ' Sm3/hr')
print('composition of gas to flare:')
printFrame(gasToFlare.getFluid())
print()
print('water to sea ', waterToSea.getFlowRate('kg/hr'), ' kg/hr')
print('composition of water to sea:')
printFrame(waterToSea.getFluid())
oiw = (1.0-waterToSea.getFluid().getComponent('water').getz()-waterToSea.getFluid().getComponent('CO2').getz())*1.0e6
print('oil in water', oiw, ' ppm mole')

gas to flare  122.33042022612489  Sm3/hr
composition of gas to flare:
                           total                   gas                       
              water   1.83365E-1            1.83365E-1        [mole fraction]
           nitrogen   2.14096E-3            2.14096E-3        [mole fraction]
                CO2   2.45746E-1            2.45746E-1        [mole fraction]
            methane   4.98614E-1            4.98614E-1        [mole fraction]
             ethane   6.10116E-2            6.10116E-2        [mole fraction]
            propane   6.67219E-3            6.67219E-3        [mole fraction]
           i-butane   9.24706E-4            9.24706E-4        [mole fraction]
           n-butane   1.07827E-3            1.07827E-3        [mole fraction]
          i-pentane    7.7464E-5             7.7464E-5        [mole fraction]
          n-pentane   1.17062E-4            1.17062E-4        [mole fraction]
           n-hexane   4.84537E-5            4.84537E-5        [mole frac